## (Structural) Node Embeddings

We consider a collections of graphs $G=(V,E)$. 
In the first step, we seek a mapping of nodes $M : V \to R^{\ell}$. We can use it to create an embedding of the entire graph by trying to encode the distribution of $n=|V|$ vectors $M(v)$, $v \in V$, associated with nodes.

Depending on the application at hand, one can use different node embeddings to produce a graph embedding. Classical graph embeddings (such as *node2vec*) are likely not what we need here.
Other methods referred to as structural graph embeddings are going to be more useful. Below we propose a few of our own, simple structural graph embedding ideas.



### Expansion properties

Suppose that we care about the expansion of neighbourhoods around nodes. For a given node $v$, we may perform BFS (up to some depth) and compute $n_i$, the number of neighbours at distance $i$ from $v$, for any $i \in \{ 1, 2, \ldots, k\}$, where $k$ is some small integer such as $k=7$. We would like to embed vertices of different degrees that expand in a similar way close to each other. Hence, we can try the following representation:

$$
E(v) = \left( \frac {n_1}{w}, \frac {n_2}{w}, \ldots, \frac {n_k}{w} \right),
$$

where $w = \sum_{i=1}^k n_i$. This, however, might not be the best choice since typically $n_i$ grows as a function of $i$ so when we compare $E(v_1)$ with $E(v_2)$ via usual distance measures, we would pay attention mostly to far neighbours (we probably should do the opposite?). For example, for good expanders (say, random graphs) we would get $(d/w, d^2/w, \ldots, d^k/w)$, where $d$ is the average degree.

Hence, it might be good to pay attention to expansion rates instead:

$$
E(v) = \left( \frac {n_1}{1 \cdot d}, \frac {n_2}{n_1 \cdot d}, \ldots, \frac {n_k}{n_{k-1} \cdot d} \right), ~~~~~~~ (1)
$$

where $d = \frac {1}{n} \sum_{v \in V} \deg(v) = \frac {2 |E|}{n}$ is the average degree. For good expanders we would get $(1,1,\ldots,1)$. 

While we investigate the different different expansion levels (which amounts to the $k$-hop ego-nets for incresasing $k$'s), we can look at some simple node features (eg. local clustering coefficient, coreness, centrality, betweenness) and enrich our feature set via some aggregation function over all nodes. This is not unlike the process performed by ReFeX.

It might also be a good idea to capture how many edges we have in the graph induced by nodes that are at distance $i$. As before, we might want to normalize it so that we measure how dense each neighbourhood is in comparison to the global density. So we might want to add (concatenate) to the vector in (1) the following additional information: 

$$
\frac {e_i}{\binom{n_i}{2} \cdot \frac {d}{n}} \approx \frac {2 e_i n}{d n_i^2}, \qquad \text{ for any } i \in \{1,2,\ldots,k\},
$$

where $e_i$ is the number of edges in the graph induced by $n_i$ nodes at distance $i$ from $v$. As usual, for good expanders (regardless of their global densities), we would see $(1,1,\ldots,1)$. 

Finally, we could pay attention to small motifs (such as triangles, short cycles, etc.) in the ego-net around $v$, and append such additional information to the embedding of $v$. For example, we can count the total number of a given motif in the entire ego-net. A more refined picture would be to compute independently, say, the average number of a given motif a node from $i$-th neighbourhood is part of. (For example, there could be more triangles far away from $v$ but not too many close to $v$. It would be good to capture this in the ebedding of $v$?) In any case, it might be good to normalize it by the expected number of such structures in a random graph with a density being the density of a given graph so that global the density does not affect the embedding vector of $v$.




### Community Based Expansion

Community structure is a good example of global structure in graphs, as opposed to local structure like degrees and triangles.
For some applications, it may be important to pay attention to communities. For example, consider two nodes that expand in a similar way so they would be embedded close to each other using the previous approaches. However, one of them has most neighbours from the same community whereas the other one has neighbours from different communities. Or maybe they have similarly looking neighbourhoods but then their neighbours can be more/less strongly connected to their respected communities. In any case, should they be embedded close to each other? Maybe not depending on the application.

Consider a partition into communities $\mathcal{A} = (A_1, A_2, \ldots, A_c)$ obtained by some stable clustering algorithm (eg. Leiden, ECG) or provided as external information. 
We may simply replace (refine) $n_i / (n_{i-1} \cdot d)$ in (1) by a sequence of $n_i^j / (n_{i-1} \cdot d)$, $j \in \{1,2,\ldots,c\}$, where $n_i^j$ is the number of nodes in the $i$-th neighbourhood that are in part $A_j$. Alternatively, for each $i \in \{1, 2, \ldots, k\}$ we can compute a vector of $n_i^j / n_i$, $j \in \{1,2,\ldots,c\}$ (fractions of vertices at distance $i$ that belong to part $A_j$) and add (concatenate) these vectors in (1) along with other things we care about. 

In the above, we only paid attention to the distribution of nodes at distance $i$ with respect to communities. In particular, we ignored information about how many nodes from community $A_j$ are adjacent to nodes from community $A_k$. If one thinks that it is of importance, then one can \emph{add} to our vector the following information: for each $i \in \{2, 3, \ldots, k\}$, we compute the fraction of edges that go from $(i-1)$-th neighbourhood to $i$-th neighbourhood with the property that they go from part $A_i$ to part $A_j$ ($i,j \in \{1, 2, \ldots, c\}$).

Similarly, it might be a good idea to capture how many edges of a certain type we have in the graph induced by nodes that are at distance $i$. As usual, we might want to normalize these counts by the expected number of edges between such nodes in either a binomial random graph with a given density or Chung-Lu graph taking into account the degrees. 
Community-based motifs could be too challenging computationally, but it makes sense to count how many triangles with all members from community $A_j$ we have, etc. Again, it might be important for some applications.

In everything we said in this sub-section, we need to make a decision if communities are interchangeable or not. If we partition users of social media into young, middle-age, and old, then we might want to distinguish such communities? Or not? In other words, if two nodes are bridges between two communities, then should they be embedded close to each other even if they bring different pairs of communities? If no, then there is no point to do anything, communities are not interchangeable. If yes, then it would be good to have some smart way to increase the chances that vectors are agnostic with respect to taking a permutation of communities. One simple (but not perfect) way would be to look at the ego-net around $v$ and count how many members in total we see from each community. Then, we call $A_1$ the most represented community, etc. 
